# Končno poročilo: Nevarnost Slovenije

## 1. Predstavitev problema

S pomočjo baze podatkov, ki opisuje pretekla kriminalna dejanja skušamo na zemljevidu prikazati čimbolj uporabne informacije o krminalu na nekem območju.
Poleg tega smo za poljubno destinacijo na slovenskem ozemlju podali oceno varnosti lokacije.

## 2. Opis podatkov

### Splošen opis
Podatke smo črpali iz strani iz https://podatki.gov.si/, bolj natančno https://podatki.gov.si/dataset/mnzpkazniva-dejanja-od-leta-2009-dalje. Uporabili smo podatke iz let 2018, 2019, 2020, 2021. Podatki iz let 2020 in 2021 v tej zbirki manjkajo, a smo jih našli na https://www.policija.si/o-slovenski-policiji/statistika/kriminaliteta.
Podatki vsebujejo kriminalna dejanja in razne podatke o njih. Mi smo uporabili stolpce: 
* UraStoritve
* DanVTednu
* OpisKD
* UpravnaEnotaStoritve
* OpisKraja

Poleg tega smo uporabili še [Polnoletni obsojenci (znani storilci) po spolu, kaznivem dejanju in glavni kazenski sankciji (tudi pogojno obsojeni), Slovenija, letno](https://podatki.gov.si/dataset/surs1360301s), [število prebivalcev po občinah](https://pxweb.stat.si/SiStatData/pxweb/sl/Data/-/2640010S.px) in [občine slovenije](https://sl.wikipedia.org/wiki/Seznam_ob%C4%8Din_v_Sloveniji).

### Obdelava podatkov
V podatkih za kazniva dejanja je nek zločin zapisan večrat. To je ocitno, saj se zaporedna številka kaznivega dejanja ponavlja večrat. Te duplikate smo odstranili zato, da so rezultati bolj točni. Pri tem smo vsekakor izgubili del podatkov, saj bi iz tega lahko pridobili tudi podatek, da se je zločin izvedel v skupini in bi ga mogoče lahko vzeli kot bolj nevarnega.

Vsakem kaznivem dejanju, ki se pojavi v podatkih, smo izračunali utež z enačbo: $$ x = ((max\_jailtime + avg\_actual\_jailtime) * dangerousness) $$
Pomen spremenljivk v enačbi:
* $max\_jailtime$: največja zaporna kazen, ki jo obsojenec lahko prejme po zakoniku. Podatek je lahko vrednost med 0 in 100, pri katerem 100 pomeni do življenska kazen.
* $avg\_actual\_jailtime$: povprečna zaporna kazen, ki so jo v preteklosti prejeli kršitelji
* $dangerousness$: subjektivnego mnenje, kako nevarno je neko dejanje za navadnega človeka. Podatek je lahko vrednost med 0 in 1.

Da je bilo določanje uteži sploh mogoče, smo morali v podatkih formatirati stolpec "OpisKD", v katerem je zapisan člen iz kazenskega zakonika tako, da je bil enak vpisu v zakoniku.  
Primer vpisa:  
* **KZ08/204-/2/S/ - TATVINA** smo spremenili v **tatvina**  

Prav tako so nekateri vpisi bili iz zastarelih zakonikov, katere smo morali najti in spremeniti tako, da so se ujemali z vpisom v zakoniku.  
Primer vpisa:  
* **KD02/103/1// - spolni napad na osebo, mlajšo od 14 let** smo spremenili v **KZ01/173/1// - spolni napad na osebo, mlajšo od petnajst let**

Uporabili smo tudi podatke o položajih regij in občin za izris na zemljevidu v aplikaciji na Androidu. Za to smo uporabili API: [*Nominatim API*](https://nominatim.org/release-docs/latest/api/Overview).

## 3. Izvedene analize

Podnaloge, ki smo jih reševali:
* Varnost posamezne regije
* Varnost posamezne občine
* Varnost posameznega prostora
* Določanje zanimivih statistik
* Vpliv dneva v tednu na nevarnost
* Vpliv ure na nevarnost

Varnost posamezne regije:
* Postopek:
  * podatke smo najprej prečistili po zgoraj vpisanem postopku
  * sestava slovarja, ki hrani uteži posameznih kaznivih dejanj
    * ```py
      kazni_utezi = dict()
      for kd, values in kazni_dict.items():
              if (pd.isna(values['average_actual_jailtime'])):
                  x = values['max_jailtime'] * 2 * values['dangerousness']
              else:
                  x = (values['max_jailtime'] + values['average_actual_jailtime']) * values['dangerousness']
              kazni_utezi[kd] = x
      ```
  * računanje nevarnosti posamezne regije
    * ```py
      regije = dict.fromkeys([str.lower(x) for x in np.unique(obcine[:, 7])], 0)
      city_errors = set()
      kd_errors_dict = defaultdict(int)


      for kd in main_data:
          city = str.lower(kd[19])
          kd = str.lower(str.strip(str.split(kd[6], '-')[-1]))
          if (city == 'neznana ue' or city == 'neznana obč' or city == 'ni podatka' or kd == 'poneverba in neupravičena uporaba tujega premoženja' or kd == ''):
              continue
          if (city == 'šentjur pri celju'):
              city = 'šentjur'
          try:
              regije[dict_obcina_regija[city]] += kazni_utezi[kd]
          except KeyError:
              if (city not in dict_obcina_regija):
                  city_errors.add(city)
                  print(f"NAPAKA PRI \"{city}\"")
              else:
                  kd_errors_dict[kd] += 1
                  print(f"NAPAKA PRI \"{kd}\"")

      kd_errors = np.array(list(kd_errors_dict.items()), dtype=object)
      kd_errors = kd_errors[np.argsort(kd_errors[:, 1])[::-1]]
      kd_errors
      ```

## 4. Rezultati